In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
import requests
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
from datetime import datetime
from datetime import timedelta
from datetime import date
import csv
import pandas as pd
import time
import json
import os

# Functions and variables

In [2]:
# Login in
USERNAME = "smussend@umd.edu"
PASSWORD = "UMDTerps2021"

# Define Computer Paths
executable_path = "C:/Users/nicho/Documents/Code/lynchings_newspapers/webscrapers/scraping_digitized_newspapers/chromedriver.exe"
download_path = r"C:\Users\nicho\Documents\Code\lynchings_newspapers\webscrapers\scraping_digitized_newspapers\downloads"

# URL
newspaperdotcomURL = "https://www.newspapers.com/"

# Define Routes

signin_field = '//*[@id="signinlink"]'
username_field = '//*[@id="username"]'
password_field = '//*[@id="password"]'

# Scraping Database Routes

column_route = "parent::node()//following-sibling::div"
# Scraping person result

person_result = "//div[contains(@id, 'search-record-Image-')]"
# Saving Photo & OCR

printSaveButton_route = '/html/body/section/div/div[1]/div[1]/div/ul[2]/li[3]/a'
saveEntirePage_route = '//*[@id="content"]/div[5]/div[2]/ul[1]/li[1]/a'
saveButton_route = '//*[@id="content"]/div[5]/div[2]/ul[2]/li[2]/a'
saveJpeg_route = '//*[@id="content"]/div[5]/div[2]/div/ul/li[1]/a'

In [3]:
def browser_instance(executable_path, headless = False, download_dir = download_path):
    chrome_options =  Options()
    if headless:
        chrome_options.add_argument("--headless")
    chrome_options.add_argument("--window-size=1920x1080")
    chrome_options.add_argument("--disable-notifications")
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--verbose')
    chrome_options.add_experimental_option("prefs", {
            "download.default_directory": download_path,
            "download.prompt_for_download": False,
            "download.directory_upgrade": True,
            "safebrowsing_for_trusted_sources_enabled": False,
            "safebrowsing.enabled": False})
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument('--disable-software-rasterizer')
    driver = webdriver.Chrome(executable_path=executable_path, options = chrome_options)
    return driver    

def search_lynching(first_name, last_name, state, lynch_date, days_delta):
    # Get dates day_delta before and after lynching
    date_before_lynching = datetime.strptime(lynch_date,"%Y-%m-%d").date() - timedelta(days=7)
    date_after_lynching = datetime.strptime(lynch_date,"%Y-%m-%d").date() + timedelta(days=7)
    
    url = f'https://www.newspapers.com/search/#query={first_name}+{last_name}&p_province=us-{state}&ymd-start={date_before_lynching}&ymd-end={date_after_lynching}'
    return url
def signIn(driver, url, signin_field, username_field, password_field):  
    driver.get(url)
    sign_in = driver.find_element_by_xpath(signin_field)
    sign_in.click()
    email_field = driver.find_element_by_xpath(username_field)
    password_field = driver.find_element_by_xpath(password_field)
    email_field.send_keys(USERNAME)
    password_field.send_keys(PASSWORD + Keys.RETURN)


def value_from_url(url):
    final_url = url[:-1].rsplit('/', 1)[-1]
    return final_url

def break_up_url(url):
    folder_dict = {}
    url = url
    for i in range(0,6):
        if i == 0:
            key = "day"
            value = url[:-1].rsplit('/', 1)[1]
            folder_dict[key] = value
            url = url[:-1].rsplit('/', 1)[0]
        else:        
            if i == 1:
                key = "month"
            if i == 2:
                key = "year"
            if i == 3:
                key = "newspaper"
            if i == 4:
                key = "city"
            if i ==5:
                key = "state"
            value = url.rsplit('/', 1)[1]
            folder_dict[key] = value
            url = url[:-1].rsplit('/', 1)[0]
    return folder_dict

def write_json_data(data_out, database_name, file_path):
        #writing data to json
        data_out = data_out
        database_name = database_name
        file = f'{file_path}/{database_name}'
        with open(file, 'w') as outfile:
            json.dump(data_out, outfile)
            
def open_json_data(file_path):    
    with open(file_path) as f:
        data = json.load(f)
    return data
            
def get_element(driver,route,word = "NA"):   
    if word != "NA":
        complete_route = f'//*[contains(text(), "{word}")]/' + route 
       
    else:
        complete_route = route
        
    element = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, complete_route)))
    
    return element

def get_list_column_entries(driver,url, word, column_route, full = True, duplicate_links = False):
    if url != "NA":
        driver.get(url)
    element_list = get_element(driver,column_route, word = word )
    element_list_url = element_list.find_elements_by_tag_name("a")
    list_elements = []
    counter = 1
    for element in element_list_url:
        if duplicate_links:
            if counter % 2 == 1: #there are two of same links, only need to grab one of them
                if full:
                    list_elements.append(element.get_attribute("href"))
                else:
                    list_elements.append(value_from_url(element.get_attribute("href")))
            counter = counter +1
        else:            
            if full:
                list_elements.append(element.get_attribute("href"))
            else:
                list_elements.append(value_from_url(element.get_attribute("href")))
    return(list_elements)

def get_person_results(results):
    newspaper_info = []
    for chunk in results:
        url = chunk.find_element_by_tag_name('a').get_attribute("href")
        newspaper_title = chunk.find_element_by_tag_name('h2').text
        other_information = chunk.find_elements_by_tag_name("p")    
        page_location = other_information[0].text
        city_state = other_information[1].text
        date = other_information[2].text
        #num_matches = other_information[3].text
        match = {"url": url, "newspaper_title": newspaper_title, "page_location": page_location, "city_state":city_state, "date": date}
        newspaper_info.append(match)
    return newspaper_info

def get_ocr_text(newspaper_urls):
    """
    input: a list of newspaper urls that link to individual photo pages
    action: changes the photo url to a ocr url
    output: saves the OCR results from newspaper.com for each photo given
    """
    result_ocr = []
    for newspaper_url in newspaper_urls:        
        ocr_string = "https://www.newspapers.com/newspage/"
        ocr_url = ocr_string + newspaper_url[33:]
        print(ocr_url)
        driver.get(ocr_url)
        ocr_text = driver.find_element_by_class_name("ocrtext").text
        match_ocr = {"url": newspaper_url, "ocr_text": ocr_text}
        result_ocr.append(match_ocr)
    return result_ocr

def save_photo(driver, url, printSaveButton_route = printSaveButton_route, saveEntirePage_route= saveEntirePage_route, saveButton_route= saveButton_route, saveJpeg_route = saveJpeg_route):
    driver.get(url)
    time.sleep(2)
    #Initial Save button
    print_save_button = get_element(driver, printSaveButton_route)        
    print_save_button.click()
    print("clicked")

    #Save Entire page
    entire_page_button = get_element(driver, saveEntirePage_route )
    entire_page_button.click()

    #Save page
    save_button = get_element(driver,saveButton_route)   
    save_button.click()

    #Save Jpeg
    save_jpeg_button = get_element(driver, saveJpeg_route)
    save_jpeg_button.click()
    time.sleep(7)
    

In [4]:
directory = "test"
path = os.path.join(download_path, directory)

print(download_path)
print(path)

C:\Users\nicho\Documents\Code\lynchings_newspapers\webscrapers\scraping_digitized_newspapers\downloads
C:\Users\nicho\Documents\Code\lynchings_newspapers\webscrapers\scraping_digitized_newspapers\downloads\test


# Scratch Pad

In [ ]:
driver = browser_instance(executable_path)
signIn(driver, url, signin_field, username_field, password_field)
time.sleep(2)
newspaper= "queen-city-times_18783"
year = "1906"
city = "agra"
state = "oklahoma"
lynch_month = 10
url_year = f"https://www.newspapers.com/browse/us/{state}/{city}/{newspaper}/{year}/"

months = get_list_column_entries(driver,url_year,"MONTH", column_route, full = False)


chosen_months = []
for month in months:
    int_month = int(month)
    if int_month == lynch_month - 1 or int_month == lynch_month+1:
        chosen_months.append(month)

urls_with_month = []        
for month in chosen_months:  
    url =  f"https://www.newspapers.com/browse/us/{state}/{city}/{newspaper}/{year}/{month}/"
    urls_with_month.append(url)

list_dates_url = []
for url in urls_with_month:
    temp = get_list_column_entries(driver, url, "DATE", column_route,  True, False)
    [list_dates_url.append(x) for x in temp]


for url in list_dates_url:
    list_pages_url = get_list_column_entries(driver, url, "PAGE", column_route,  True, False) 
    url_dict = break_up_url(url)
    directory = f'{url_dict["state"]}_{url_dict["city"]}_{url_dict["newspaper"]}_{url_dict["year"]}_{url_dict["month"]}'
    path = os.path.join(download_path, directory)
    print(path)
    print(type(path))
    os.mkdir(path)
    driver.quit()    
    for page in list_pages_url:        
        driver2 = browser_instance(executable_path, download_dir = f'{download_path}\{directory}' )        
        signIn(driver2, url, signin_field, username_field, password_field)
        time.sleep(2)
        save_photo(driver2, page)
    



In [195]:
f'{download_path}'   

'C:\\Users\\nicho\\Documents\\Code\\lynchings_newspapers\\webscrapers\\scraping_digitized_newspapers\\downloads'

In [183]:
print(type("hi"))

<class 'str'>


In [171]:


folder_dict = {}
url = "https://www.newspapers.com/browse/us/oklahoma/agra/queen-city-times_18783/1906/09/20/"
for i in range(0,6):
    if i == 0:
        key = "day"
        value = url[:-1].rsplit('/', 1)[1]
        folder_dict[key] = value
        url = url[:-1].rsplit('/', 1)[0]
    else:        
        if i == 1:
            key = "month"
        if i == 2:
            key = "year"
        if i == 3:
            key = "newspaper"
        if i == 4:
            key = "city"
        if i ==5:
            key = "state"
        value = url.rsplit('/', 1)[1]
        folder_dict[key] = value
        url = url[:-1].rsplit('/', 1)[0]
return folder_dict
    

        

20


In [167]:
url = "https://www.newspapers.com/browse/us/oklahoma/agra/queen-city-times_18783/1906/09/20"
print(url.rsplit('/', 1)[1])

20


In [36]:
df_json = open_json_data("newspaperdotcom_newjersey_ohio.json")


list

In [78]:
result = []
states_list = ["new-jersey"]
cities_list = ["allentown"]
years_list = ["1956"]
for state_entry in df_json:    
    if state_entry["state"] in states_list:
        city_entry = state_entry["city"]        
        for city, newspapers in city_entry.items(): 
            if city in cities_list:
                for newspaper, years in newspapers.items():
                    #print(newspaper, "-->", years)
                    if any(item in years for item in years_list):
                        pass
                        #print("======")
                        print(newspaper, "-->", years)
                    
        
       
        
        

allentown-messenger_9256 --> ['1903', '1904', '1905', '1906', '1907', '1908', '1909', '1910', '1911', '1912', '1913', '1914', '1915', '1916', '1917', '1918', '1919', '1920', '1921', '1922', '1923', '1924', '1925', '1926', '1927', '1928', '1929', '1930', '1931', '1932', '1933', '1934', '1935', '1936', '1937', '1938', '1939', '1940', '1941', '1942', '1943', '1944', '1945', '1946', '1947', '1948', '1949', '1950', '1951', '1952', '1953', '1954', '1955', '1956', '1957', '1958', '1959', '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968', '1969', '1970', '1971']


In [39]:
test = {key:value for key,value in df_json.items() if key in ["new-jersey]"]}

AttributeError: 'list' object has no attribute 'items'

In [ ]:
print(df_json)

In [31]:
lynch_month = 10
chosen_months = []
for month in months:
    int_month = int(month)
    if int_month == lynch_month - 1 or int_month == lynch_month+1:
        chosen_months.append(month)
print(chosen_months)
    


['09', '11']


# Person Scraper

## Sign in 

In [5]:
driver = browser_instance(executable_path, False)
signIn(driver, newspaperdotcomURL, signin_field, username_field, password_field)
time.sleep(3)
driver.get(search_lynching("luther", "holbert", "ms","1904-02-01", 7))

## Search

### Person Results

In [6]:
results = driver.find_elements_by_xpath(person_result)
newspaper_info = get_person_results(results)
df_newspaper_info = pd.DataFrame(newspaper_info)


### Save Photo

In [7]:
for newspaper in newspaper_info:
    url = newspaper["url"]    
    save_photo(driver, url)


clicked
clicked
clicked
clicked
clicked
clicked


KeyboardInterrupt: 

### SAVE OCR Text

In [ ]:
newspaper_urls = []
for newspaper in newspaper_info:
    newspaper_urls.append(newspaper["url"])

result_ocr = get_ocr_text(newspaper_urls)
df_result_ocr = pd.DataFrame(result_ocr)    
    

### Dataframe

In [ ]:
df_newspaper_result = pd.merge(df_result_ocr, df_newspaper_info)
df_newspaper_result.to_csv("df_newspaper_result.csv")